# Chest-prediction

## PRELIMINARY ACTION !

⚠️ Please go to ➤ https://drive.google.com/file/d/1lLrHbpUQE-Kd-jZ68Uk7SFwawbzqf6Av/view?usp=drive_link

and download the dataset.

Put the zip file into your "*raw_data*" folder

## data loading

In [65]:
import numpy as np
import pandas as pd
import pdb

import os
from pathlib import Path
from PIL import Image

### Loading images

In [66]:
PROJECT_NAME = "chest-predictor" #to be adapted depending on the name of the Project Name in your system
NUMBER_OF_IMAGES = 3000 # nbr of images to be loaded or 'full' to load the entire dataset (+100k images)

In [67]:
USERNAME = os.environ.get('USER')

In [68]:
LOCAL_DATA_PATH = Path(f"/Users/{USERNAME}/code/sachamagier/{PROJECT_NAME}/raw_data/resized_dataset")

In [69]:
print(f"LOCAL_DATA_PATH: {LOCAL_DATA_PATH}")

LOCAL_DATA_PATH: /Users/rvnmll/code/sachamagier/chest-predictor/raw_data/resized_dataset


In [70]:
def loading_data():
    """This function either get all the images if the user set NUMBER_OF_IMAGES
    to 'full' or the number of imgaes otherwise """

    images_data = []

    # Define the path to the folder
    folder_path = f'../raw_data/resized_dataset/images/set_full/'


    # Get a list of all files in the folder
    file_list = os.listdir(folder_path)

    # Filter the list to only include image files
    image_files = [f for f in file_list if f.endswith('.png') or f.endswith('.jpg') or f.endswith('.jpeg')]


    # Loop through the first NUMBER_OF_IMAGES
    for i, image_file in enumerate(image_files):

        # Stop the loop after NUMBER_OF_IMAGES iterations
        if i == NUMBER_OF_IMAGES:
            break

        # Open the image file
        with Image.open(folder_path + image_file) as image:
            # Add the image to the list
            images_data.append((image_file, np.array(image)))

    return images_data

In [71]:
import urllib.request
import zipfile
import shutil


if LOCAL_DATA_PATH.is_dir():
    print("Load local data...")
    # loading data into data
    images_data = loading_data()
else:
    print("Unziping file and loading the data...")

    output_path = "../raw_data/resized_dataset.zip"
    # unzip the file
    with zipfile.ZipFile(output_path, "r") as zip_ref:
        for file_info in zip_ref.infolist():
            zip_ref.extract(file_info, "../raw_data/")

    if Path("../raw_data/__MACOSX").is_dir():
        # remove the __MACOSX folder if it exists
        shutil.rmtree("../raw_data/__MACOSX")

    # remove the zip file
    os.remove(output_path)
    images_data = loading_data()

print("data loaded.")

Unziping file and loading the data...
data loaded.


In [72]:
# Create a dataframe from the list of images and their indices
images_df = pd.DataFrame(images_data, columns=['Image Index', 'image'])

# Set the index of the dataframe to the 'Image Index' column
images_df = images_df.set_index('Image Index').sort_index(ascending=True)

In [73]:
images_df.shape

(3000, 1)

In [74]:
images_df['image'][0].shape

/tmp/ipykernel_20441/4042123707.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  images_df['image'][0].shape


(256, 256)

### Loading labels data

In [75]:
labels_df = pd.read_csv('../raw_data/resized_dataset/Data_Entry_2017.csv')

In [76]:
labels_df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN


In [77]:
unique_lab = labels_df['Finding Labels'].unique()

In [78]:
len(unique_lab)

836

### Merging Images with labels and creating a new DF

In [79]:
# Merge the image_df and labels_df dataframes on the 'Image Index' column
merged_df = pd.merge(images_df, labels_df[['Image Index', 'Finding Labels']], left_index=True, right_on='Image Index', how='inner')

# Rename the 'Finding Labels' column to 'labels'
merged_df = merged_df.rename(columns={'Finding Labels': 'labels'})

# Set the index of the dataframe to the 'Image Index' column
merged_df = merged_df.set_index('Image Index').sort_index(ascending=True)

In [80]:
merged_df.head()

,image,labels
Image Index,,
00000005_003.png,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",No Finding
00000006_000.png,"[[45, 43, 38, 34, 30, 27, 24, 21, 19, 17, 16, ...",No Finding
00000013_031.png,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",Emphysema|Mass
00000017_002.png,"[[14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, ...",No Finding
00000018_000.png,"[[32, 30, 26, 24, 24, 23, 23, 22, 22, 22, 21, ...",No Finding


### droping the rows with images of shape (256, 256, 4)

In [81]:
# Find the images with shape (256, 256, 4)
images_with_shape_4 = [img for img in merged_df['image'] if np.shape(img) == (256, 256, 4)]

# Print the number of images with shape (256, 256, 4)
print(len(images_with_shape_4))


9


In [82]:
# Find the indices of the images with shape (256, 256, 4)
indices_to_drop = merged_df[merged_df['image'].apply(lambda x: np.shape(x) == (256, 256, 4))].index

# Drop the rows with the images with shape (256, 256, 4)
merged_df = merged_df.drop(indices_to_drop)

In [83]:
# Find the images with shape (256, 256, 4)
images_with_shape_4 = [img for img in merged_df['image'] if np.shape(img) == (256, 256, 4)]

# Print the number of images with shape (256, 256, 4)
print(len(images_with_shape_4))

0


### Encoding labels

In [84]:
# Define the list of labels
labels = ['Atelectasis', 'Consolidation', 'Infiltration', 'Pneumothorax', 'Edema',
           'Emphysema', 'Fibrosis', 'Effusion', 'Pneumonia', 'Pleural_Thickening',
           'Cardiomegaly', 'Nodule', 'Mass', 'Hernia', 'No Finding']

# Create a new dataframe with one-hot encoded columns for the labels
one_hot_df = merged_df['labels'].str.get_dummies(sep='|')

# Concatenate the one-hot encoded dataframe with the original dataframe
merged_df = pd.concat([merged_df, one_hot_df], axis=1)


In [85]:
merged_df.columns

Index(['image', 'labels', 'Atelectasis', 'Cardiomegaly', 'Consolidation',
       'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration',
       'Mass', 'No Finding', 'Nodule', 'Pleural_Thickening', 'Pneumonia',
       'Pneumothorax'],
      dtype='object')

## Creating 'X' and 'y'

In [86]:
y = one_hot_df

In [87]:
X = merged_df['image']

In [88]:
X.info(memory_usage='deep')

<class 'pandas.core.series.Series'>
Index: 2991 entries, 00000005_003.png to 00030797_000.png
Series name: image
Non-Null Count  Dtype 
--------------  ----- 
2991 non-null   object
dtypes: object(1)
memory usage: 187.5 MB


In [89]:
X = np.array([np.reshape(img, (256, 256, 1)) for img in X])

In [90]:
X.shape

(2991, 256, 256, 1)

## Creating a simple model

In [101]:
#Imports
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Rescaling
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [92]:
##### Splitting X and y in train/val/test #####

# First split: split data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)

# Second split: split training data into training and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=69)

In [93]:
#Image parameters
IMG_HEIGHT = 256
IMG_WIDTH = 256
BATCH_SIZE = 32
NUM_CLASSES = 15

In [120]:
#import metrics
from sklearn.metrics import accuracy_score, hamming_loss, precision_score, recall_score

# Some extra metrics that we'll use to evaluate our model
def EMR(y_pred, y_test):
    """ Exact Match ratio, this takes the ratio of exact matches per samble.
    A prediction of a sample that accurately predicts ALL lables will be considered
    an Exact match, we cound all the exact matches and divide them by the sample
    amount to get the EM ratio
    """
    return np.all(y_pred == y_test, axis=1).mean()

def hamming_score_value(y_pred, y_test):
    """Proportion of the predicted correct labels to the total number (predicted and actual) of labels
    for that instance. Overall accuracy is the average across all instances.
    We use the hamming loss because it's the opposite of the hamming score, it reports
    how many times on average, the relevance of an example to a class label is incorrectly predicted.
    """
    return 1 - hamming_loss(y_pred, y_test)

def average_precision(y_pred, y_test):
    return precision_score(y_test, y_pred, average='samples')
def average_recall(y_pred, y_test):
    return recall_score(y_test, y_pred, average='samples')

def evaluating_model(y_pred, y_test):
    print('Exact Match Ratio:', EMR(y_pred, y_test)),
    print('Hamming Score:', hamming_score_value(y_pred, y_test)),
    print('Average Precision:', average_precision(y_pred, y_test)),
    print('Average Recall:', average_recall(y_pred, y_test))


In [99]:
# Define a simple model
def simple_model():

    model = Sequential()
    model.add(Rescaling(1./255, input_shape=(IMG_HEIGHT,IMG_WIDTH,1)))

    model.add(Conv2D(16, kernel_size=10, activation='relu'))
    model.add(MaxPooling2D(3))

    model.add(Conv2D(32, kernel_size=8, activation="relu"))
    model.add(MaxPooling2D(3))

    model.add(Conv2D(32, kernel_size=6, activation="relu"))
    model.add(MaxPooling2D(3))

    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(NUM_CLASSES, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [104]:
models_dir = '../models' #This is where the best models will be saved

# Ensure that the directory exists
os.makedirs(models_dir, exist_ok=True)

es = EarlyStopping(monitor='val_loss',
                   patience=10,
                   verbose=0,
                   restore_best_weights=True)

model_checkpoint = ModelCheckpoint(filepath= os.path.join(models_dir, 'simple_model_best.h5'),
                                   save_best_only=True,
                                   monitor='val_loss',
                                   mode='min')


In [105]:
s_model = simple_model()
s_model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_8 (Rescaling)     (None, 256, 256, 1)       0         
                                                                 
 conv2d_24 (Conv2D)          (None, 247, 247, 16)      1616      
                                                                 
 max_pooling2d_24 (MaxPooli  (None, 82, 82, 16)        0         
 ng2D)                                                           
                                                                 
 conv2d_25 (Conv2D)          (None, 75, 75, 32)        32800     
                                                                 
 max_pooling2d_25 (MaxPooli  (None, 25, 25, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_26 (Conv2D)          (None, 20, 20, 32)       

In [106]:
# Train the model
history = s_model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=BATCH_SIZE,
    callbacks=[es, model_checkpoint])

Epoch 1/10
60/60 [==============================] - 14s 203ms/step - loss: 0.2623 - accuracy: 0.5180 - val_loss: 0.2230 - val_accuracy: 0.5511
Epoch 2/10


/home/rvnmll/.pyenv/versions/3.10.6/envs/chest-predictor/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


60/60 [==============================] - 13s 216ms/step - loss: 0.2239 - accuracy: 0.5311 - val_loss: 0.2209 - val_accuracy: 0.5511
Epoch 3/10
60/60 [==============================] - 12s 207ms/step - loss: 0.2242 - accuracy: 0.5306 - val_loss: 0.2203 - val_accuracy: 0.5511
Epoch 4/10
60/60 [==============================] - 13s 214ms/step - loss: 0.2218 - accuracy: 0.5248 - val_loss: 0.2212 - val_accuracy: 0.5511
Epoch 5/10
60/60 [==============================] - 15s 242ms/step - loss: 0.2197 - accuracy: 0.5306 - val_loss: 0.2210 - val_accuracy: 0.5511
Epoch 6/10
60/60 [==============================] - 15s 251ms/step - loss: 0.2165 - accuracy: 0.5243 - val_loss: 0.2193 - val_accuracy: 0.5511
Epoch 7/10
60/60 [==============================] - 15s 258ms/step - loss: 0.2163 - accuracy: 0.5233 - val_loss: 0.2195 - val_accuracy: 0.5073
Epoch 8/10
60/60 [==============================] - 16s 269ms/step - loss: 0.2151 - accuracy: 0.5227 - val_loss: 0.2171 - val_accuracy: 0.5386
Epoch 9/10

In [117]:
test_loss, test_accuracy = s_model.evaluate(X_test, y_test)
val_loss, val_accuracy = s_model.evaluate(X_val, y_val)
print('val accuracy:', val_accuracy)
print('test accuracy:', test_accuracy)

15/15 [==============================] - 1s 63ms/step - loss: 0.2193 - accuracy: 0.5470
val accuracy: 0.5469728708267212
test accuracy: 0.5141903162002563


In [111]:
y_pred = s_model.predict(X_test)

19/19 [==============================] - 1s 66ms/step


In [112]:
y_pred[0]

array([0.1611082 , 0.08431179, 0.04302759, 0.0482697 , 0.23415886,
       0.0681626 , 0.03675291, 0.01000772, 0.30514473, 0.14327462,
       0.501417  , 0.10649241, 0.03310282, 0.00968514, 0.08492978],
      dtype=float32)

In [113]:
y.head()

,Atelectasis,Cardiomegaly,Consolidation,Edema,Effusion,Emphysema,Fibrosis,Hernia,Infiltration,Mass,No Finding,Nodule,Pleural_Thickening,Pneumonia,Pneumothorax
Image Index,,,,,,,,,,,,,,,
00000005_003.png,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
00000006_000.png,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
00000013_031.png,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
00000017_002.png,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
00000018_000.png,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [121]:
#Because we need binary results to evaluate the model we'll put a threshold
#that will transform the values to either 0 or 1

threshold = 0.5
y_pred_binary = (y_pred >= threshold).astype(int)

print(evaluating_model(y_pred_binary, y_test))

Exact Match Ratio: 0.44574290484140233
Hamming Score: 0.9225375626043406
Average Precision: 0.44741235392320533
Average Recall: 0.44629938786866996
None


/home/rvnmll/.pyenv/versions/3.10.6/envs/chest-predictor/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## We train a more complex model to see how our scores change 

In [137]:
# Define a more complex model
def model():

    model = Sequential()
    model.add(Rescaling(1./255, input_shape=(IMG_HEIGHT,IMG_WIDTH,1)))

    model.add(Conv2D(64, kernel_size=3, activation='relu'))
    model.add(MaxPooling2D(2))

    model.add(Conv2D(64, kernel_size=3, activation='relu'))
    model.add(MaxPooling2D(2))

    model.add(Conv2D(32, kernel_size=5, activation='relu'))
    model.add(MaxPooling2D(2))

    model.add(Conv2D(32, kernel_size=5, activation='relu'))
    model.add(MaxPooling2D(2))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))

    model.add(Dense(NUM_CLASSES, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [138]:
model_checkpoint = ModelCheckpoint(filepath= os.path.join(models_dir, 'better_model.h5'),
                                   save_best_only=True,
                                   monitor='val_loss',
                                   mode='min')


In [139]:
model = model()
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_13 (Rescaling)    (None, 256, 256, 1)       0         
                                                                 
 conv2d_40 (Conv2D)          (None, 254, 254, 64)      640       
                                                                 
 max_pooling2d_40 (MaxPooli  (None, 127, 127, 64)      0         
 ng2D)                                                           
                                                                 
 conv2d_41 (Conv2D)          (None, 125, 125, 64)      36928     
                                                                 
 max_pooling2d_41 (MaxPooli  (None, 62, 62, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_42 (Conv2D)          (None, 58, 58, 32)      

In [140]:
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=BATCH_SIZE,
    callbacks=[es, model_checkpoint])

Epoch 1/10
60/60 [==============================] - 55s 795ms/step - loss: 0.2745 - accuracy: 0.5280 - val_loss: 0.2245 - val_accuracy: 0.5511
Epoch 2/10


/home/rvnmll/.pyenv/versions/3.10.6/envs/chest-predictor/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


60/60 [==============================] - 53s 875ms/step - loss: 0.2249 - accuracy: 0.5311 - val_loss: 0.2205 - val_accuracy: 0.5511
Epoch 3/10
60/60 [==============================] - 58s 977ms/step - loss: 0.2232 - accuracy: 0.5311 - val_loss: 0.2204 - val_accuracy: 0.5511
Epoch 4/10
60/60 [==============================] - 66s 1s/step - loss: 0.2214 - accuracy: 0.5311 - val_loss: 0.2181 - val_accuracy: 0.5511
Epoch 5/10
60/60 [==============================] - 68s 1s/step - loss: 0.2182 - accuracy: 0.5290 - val_loss: 0.2176 - val_accuracy: 0.5428
Epoch 6/10
60/60 [==============================] - 63s 1s/step - loss: 0.2181 - accuracy: 0.5259 - val_loss: 0.2185 - val_accuracy: 0.5511
Epoch 7/10
60/60 [==============================] - 64s 1s/step - loss: 0.2176 - accuracy: 0.5254 - val_loss: 0.2165 - val_accuracy: 0.5470
Epoch 8/10
60/60 [==============================] - 64s 1s/step - loss: 0.2143 - accuracy: 0.5274 - val_loss: 0.2197 - val_accuracy: 0.5511
Epoch 9/10
60/60 [=======

In [141]:
y_pred = model.predict(X_test)

19/19 [==============================] - 4s 193ms/step


In [142]:
threshold = 0.5
y_pred_binary = (y_pred >= threshold).astype(int)

print(evaluating_model(y_pred_binary, y_test))

Exact Match Ratio: 0.42070116861435725
Hamming Score: 0.9257651641624931
Average Precision: 0.42070116861435725
Average Recall: 0.42070116861435725
None


/home/rvnmll/.pyenv/versions/3.10.6/envs/chest-predictor/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
